# Лаба 2

**Дедлайн**: 25 ноября

**Задача**: написать определитель тональности текста (сообщениея в Twitter) c помощью fine-tuning-а на датасете RuSentiTweet (https://github.com/sismetanin/rusentitweet)

На что обратить внимание:
* Подготовка данных (очистка, токенизация и упаковка датасета в удобный класс) - у вас в задании другой датасет, соответственно обработка может поменяться. В датасете несколько файлов, скачайте rusentitweet_full.csv и работайте с ним
* Процедура дообучения. Вам необходимо доработать имеющуюся процедуру:
    * Добавить графики качества обучения модели в зависимости от шага (делать валидацию каждые 100 шагов (например), а не раз в эпоху)
    * Замерить время обучения
    * Добавить больше метрик для отслеживания (изучите по открытым источникам, какие метрики используются для задачи определения тональности и почему)
    * Добавить заморозку части слоев (все, кроме слоя классификации, или кроме слоя классификации + 2-3 последних слоев с интентами)
    * Подобрать количество эпох, размер батча и заморозку так, чтобы модель давала лучший результат
* Модель для дообучения (попробуйте как минимум 2 разных модели), искать подходящие модели можно с помощью гугла и https://huggingface.co/
* Результаты **всех** экспериментов должны быть описаны в отдельной ячейке
* Inference модели - обученную модель нужно обернуть в удобную функцию для использования, которая по тексту будет возвращать его тональность



# Imports

Устанавливаем и подключаем необходимые библиотеки

In [1]:
# !pip install transformers

In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
tqdm.pandas()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

# Загрузка и обработка данных

В примере используется датасет https://www.kaggle.com/datasets/blackmoon/russian-language-toxic-comments, нужно поменять его на тот, что указан выше

In [3]:
df = pd.read_csv("labeled.csv")
# Обратите внимание, что для корректной работы BertForSequenceClassification
# метки классов должны быть в виде целого числа из промежутка (0, 1, ..., num_classes - 1)
# обязательно (!) типа int, а не float, str и т.д.
df["toxic"] = df["toxic"].astype(int)
df.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1
1,"Хохлы, это отдушина затюканого россиянина, мол...",1
2,Собаке - собачья смерть\n,1
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1


In [4]:
# Проверим количество классов - если больше 3-х (позитив, негатив и нейтральный)
# или двух (позитив и негатив), то нужно убрать лишние или привести к описанной шкале
# (все градации негатива свернуть в один класс)
df["toxic"].unique()

array([1, 0])

**Функция для обработки текста**

Поддерживает следующие этапы:
1. Приведение к lowercase
2. Удаление спецсимволов

Возможные улучшения:
1. Обработка смайликов
2. Замена цифр на слова (например, "5" на "пять")
3. Удаление пунктуации (зависит от используемой базовой модели)
4. Убрать приведение к lowercase, если модель позволяет

In [5]:
def clean_text(text):
    # Небольшой совет:
    # Если тип параметра функции неизменяемый (immutable) - например, int, float, str, tuple,
    # то он передается "по значению" - в функции создается новый объект и его можно изменять.
    # Поэтому мы можем менять значение параметра text и это не повлечет за собой изменение
    # внешнего объекта
    # Если же тип параметра изменяемый (mutable) - list, set, dict, DataFrame и т.д.,
    # то рекомендуется создать переменную-копию (new_text, например) и изменять ее

    text = text.lower()
    text = text.replace("\n", "")

    return text

**Подготовим датасет**

In [6]:
# Маленькая красота - визуализация прогресса выполнения построчных операций
# Для этого используем progress_apply и в начало добавили 
# from tqdm.notebook import tqdm
# tqdm.pandas()

df["comment_cleaned"] = df["comment"].progress_apply(clean_text)

  0%|          | 0/14412 [00:00<?, ?it/s]

,comment,toxic,comment_cleaned
0,"Верблюдов-то за что? Дебилы, бл...\n",1,"верблюдов-то за что? дебилы, бл..."
1,"Хохлы, это отдушина затюканого россиянина, мол...",1,"хохлы, это отдушина затюканого россиянина, мол..."
2,Собаке - собачья смерть\n,1,собаке - собачья смерть
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1,"страницу обнови, дебил. это тоже не оскорблени..."
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1,"тебя не убедил 6-страничный пдф в том, что скр..."
5,Для каких стан является эталоном современная с...,1,для каких стан является эталоном современная с...
6,В шапке были ссылки на инфу по текущему фильму...,0,в шапке были ссылки на инфу по текущему фильму...
7,УПАД Т! ТАМ НЕЛЬЗЯ СТРОИТЬ! ТЕХНОЛОГИЙ НЕТ! РА...,1,упад т! там нельзя строить! технологий нет! ра...
8,"Ебать тебя разносит, шизик.\n",1,"ебать тебя разносит, шизик."
9,"Обосрался, сиди обтекай\n",1,"обосрался, сиди обтекай"


In [7]:
# Загружаем токенайзер - по имени модели на huggingface hub-е

tokenizer = BertTokenizer.from_pretrained(
    "DeepPavlov/rubert-base-cased",
    do_lower_case = True
)

In [8]:
def tokenize(text):
    res = tokenizer.encode_plus(
        text,
        add_special_tokens=True,  # Да, добавляем, т.к. дальше даем на вход BERT-у
        max_length=64,  # Максимальная длина входной последовательности - позволяет оптимизировать память
                        # Ограничение BERT-a - 512, но если сделать меньше, то модель
                        # будет обучаться быстрее
                        # Можно заранее посчитать максимальную длину последовательности 
                        # на датасете (считать нужно в токенах по attention mask)
        truncation=True,
        padding='max_length',
        # pad_to_max_length=True, # Нужно ли дополнять предложение до максимальной длины
                                 # Да, нужно - в таком случае можно делить на батчи
                                 # (если векторы будут разной размерности, упадем с ошибкой)
        return_attention_mask=True,  # Attention mask - показывает, имеет ли токен смысл
                                     # токен [PAD] - 0, остальные - 1 
        return_tensors="pt"  # Указываем тип тензоров, нам нуше PyTorch
    )
    return pd.Series([res["input_ids"], res["attention_mask"]])

df[["input_ids", "attention_mask"]] = df["comment_cleaned"].progress_apply(tokenize)

  0%|          | 0/14412 [00:00<?, ?it/s]

In [9]:
test_size = 0.3
batch_size = 32

# Делим выборку на трейн и тест со стратификацией - сохраняя распределение классов
train_df, test_df = train_test_split(
    df,
    test_size=test_size,
    shuffle=True,
    stratify=df["toxic"].values
)

# Train and validation sets
train_set = TensorDataset(torch.cat(list(train_df["input_ids"].values), dim=0),
                          torch.cat(list(train_df["attention_mask"].values), dim=0), 
                          torch.tensor(train_df["toxic"].values))

test_set = TensorDataset(torch.cat(list(test_df["input_ids"].values), dim=0),
                         torch.cat(list(test_df["attention_mask"].values), dim=0), 
                         torch.tensor(test_df["toxic"].values))

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            batch_size=batch_size
        )

test_dataloader = DataLoader(
            test_set,
            batch_size=batch_size
        )

# Обучаем модель

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [11]:
model = BertForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [12]:
epochs = 10

optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr = 5e-6,
    eps = 1e-08
)

for _ in tqdm(range(epochs), desc='Epoch'):
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        optimizer.zero_grad()

        b_labels = b_labels.type(torch.LongTensor)
        b_labels = b_labels.to(device)

        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        
        # Backward pass
        train_output.loss.backward()
        optimizer.step()

        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_f1 = []

    for batch in tqdm(test_dataloader, total=len(test_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
          
        logits = eval_output.logits
        y_pred = torch.argmax(logits, dim = -1)
        
        y_pred = y_pred.detach().cpu().numpy()
        y_true = b_labels.to('cpu').numpy()
        
        # Calculate validation metrics
        val_f1_value = f1_score(y_true, y_pred, average='macro')
        val_f1.append(val_f1_value)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation F1-score: {:.4f}'.format(sum(val_f1)/len(val_f1)))

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/316 [00:00<?, ?it/s]

  0%|          | 0/136 [00:00<?, ?it/s]


	 - Train loss: 0.3478
	 - Validation F1-score: 0.8941


  0%|          | 0/316 [00:00<?, ?it/s]

...

  0%|          | 0/136 [00:00<?, ?it/s]


	 - Train loss: 0.0427
	 - Validation F1-score: 0.9049


  0%|          | 0/316 [00:00<?, ?it/s]

  0%|          | 0/136 [00:00<?, ?it/s]


	 - Train loss: 0.0394
	 - Validation F1-score: 0.9043


# Применение (inference) модели

Ниже напишите функцию, которая будет получать на вход текст, а на выходе писать его тональность

In [16]:
model.eval()
# len(test_dataloader)

def predict(text):
    text = clean_text(text)
    tokenized = tokenize(text)
    tokenized[0] = tokenized[0].to(device)
    tokenized[1] = tokenized[1].to(device)
    evaluated = model(tokenized[0], token_type_ids= None, attention_mask=tokenized[1])
    print(torch.argmax(evaluated.logits, dim = -1).detach().cpu().numpy())

In [15]:
torch.save(model.state_dict(), './savedModel')

In [37]:
predict("бурзум")

[1]
